# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [1]:
!nvidia-smi

Thu May 20 08:39:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
# your workspace in your drive
workspace = 'ML2021-hw7'

try:
  os.chdir(os.path.join('/content/gdrive/MyDrive/', workspace))
except:
  os.makedirs(os.path.join('/content/gdrive/MyDrive/', workspace), exist_ok=True)
  os.chdir(os.path.join('/content/gdrive/MyDrive/', workspace))

In [4]:
# Download link 1
#!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

#!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
#!nvidia-smi

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [5]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 16.2MB/s 
     |████████████████████████████████| 901kB 53.5MB/s 
     |████████████████████████████████| 3.3MB 65.2MB/s 


## Import Packages

In [6]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizer, get_linear_schedule_with_warmup, BertModel, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [7]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 51kB 3.3MB/s 


## Load Model and Tokenizer




 

In [8]:
tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-large")
model = BertForQuestionAnswering.from_pretrained("hfl/chinese-macbert-large").to(device)

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Some weights of the model checkpoint at hfl/chinese-macbert-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [9]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

In [10]:
print(tokenizer.tokenize('✔', add_special_tokens=False))
print(tokenizer.tokenize('●', add_special_tokens=False))
print(tokenizer.tokenize('✦', add_special_tokens=False))
print(tokenizer.tokenize('☺', add_special_tokens=False))
print(tokenizer.tokenize('☆', add_special_tokens=False))

# 取代' ' \u200b \u200e \u3000 # 是為了讓tokenize前後index一致
# 用✔ ● ✦ ☺ ☆ 當佔位符，沒有意義
#train_paragraphs = [i.replace(' ','✔').replace('\u200b','✦').replace('\u200e', '☺').replace('\u3000', '☆').replace('#','●') for i in train_paragraphs]
dev_paragraphs = [i.replace(' ','✔').replace('\u200b','✦').replace('\u200e', '☺').replace('\u3000', '☆').replace('#','●') for i in dev_paragraphs]
test_paragraphs = [i.replace(' ','✔').replace('\u200b','✦').replace('\u200e', '☺').replace('\u3000', '☆').replace('#','●') for i in test_paragraphs]

['✔']
['●']
['✦']
['☺']
['☆']


## Tokenize Data

In [11]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [12]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 50
        self.max_paragraph_len = 300
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 200

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            #mid = int((answer_start_token + answer_end_token) // (2+random.uniform(-1,1)))
            #paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            if answer_start_token >  self.max_paragraph_len:
                # paragraph_start 不能從0開始
                paragraph_start = min(answer_start_token - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len)
            else:
                paragraph_start = 0
            paragraph_end = paragraph_start + self.max_paragraph_len
            

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [13]:
def index_before_tokenize(tokens, start, end):
    char_count, new_start, new_end = 0, 512, 512
    start_flag = 0
    end_flag = 0
        
    for i, token in enumerate(tokens):
        if token == '[UNK]' or token == '[CLS]' or token == '[SEP]':
            if i == start:
                new_start = char_count
            if i == end:
                new_end = char_count
            char_count += 1
        else:
            for c in token:
                if i == start and start_flag == 0:
                    #print(token)
                    new_start = char_count
                    start_flag = 1
                if i == end:
                    #print(token)
                    new_end = char_count
                    end_flag = 1
                if c != '#':
                    char_count += 1
    return new_start, new_end

In [14]:
def evaluate(data, output, doc_stride=150, paragraph=None, paragraph_tokenized=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    # index in the whole tokens (not just relative to window)
    entire_start_index = 0
    entire_end_index = 0
    
    for k in range(num_of_windows):
        #print('window',k)
        # Obtain answer by choosing the most probable start position / end position
        mask = data[1][0][k].bool() &  data[2][0][k].bool() # token type & attention mask
        masked_output_start = torch.masked_select(output.start_logits[k], mask)[:-1] # -1 is [SEP]
        start_prob, start_index = torch.max(masked_output_start, dim=0)
        #masked_output_end = torch.masked_select(output.end_logits[k], mask)[start_index:-1] # -1 is [SEP]
        masked_output_end = torch.masked_select(output.end_logits[k], mask)[:-1] # -1 is [SEP]
        end_prob, end_index = torch.max(masked_output_end, dim=0)
        #end_index += start_index 
        

        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        masked_data = torch.masked_select(data[0][0][k], mask)[:-1] # -1 is [SEP]

        # Replace answer if calculated probability is larger than previous windows
        if (prob > max_prob) and (end_index - start_index <= 30) and (end_index > start_index):
            max_prob = prob
            entire_start_index = start_index.item() + doc_stride * k
            entire_end_index = end_index.item() + doc_stride * k
            #print('entire_start_index',entire_start_index)
            #print('entire_end_index',entire_end_index)
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(masked_data[start_index : end_index + 1])
            # Remove spaces in answer (e.g. "大 金" --> "大金")
            answer = answer.replace('✔', ' ').replace('✦','\u200b').replace('☺','\u200e').replace('☆','\u3000').replace('●','#').replace(' ','')

    
    # if [UNK] in prediction, use orignal span of paragrah
    if '[UNK]' in answer:
        print('found [UNK] in prediction, using original text')
        print('original prediction', answer)
        # find the index of answer in the orinal paragrah

        new_start, new_end = index_before_tokenize(tokens=paragraph_tokenized, 
                                                   start=entire_start_index, end=entire_end_index)
        #print('new_start',new_start)
        #print('new_end',new_end)
        answer = paragraph[new_start:new_end+1]
        answer = answer.replace('✔', ' ').replace('✦','\u200b').replace('☺','\u200e').replace('☆','\u3000').replace('●','#')
        print('final prediction',answer)

            
    
    return answer

## Training

In [15]:
num_epoch = 3
validation = False
logging_step = 100
learning_rate = 1e-5
doc_stride = 200
# batch accumulation parameter
# accum_iter = 4  

optimizer = AdamW(model.parameters(), lr=learning_rate)

if not validation:
    dev_set = QA_Dataset("train", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
    train_set = torch.utils.data.ConcatDataset([train_set, dev_set])
    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

# Total number of training steps
total_steps = len(train_loader) * num_epoch
print('total_steps', total_steps)
# Set up the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps= 0, # Default value
                                                num_training_steps=total_steps)


    
model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for idx, data in enumerate(tqdm(train_loader)):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])


        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        # normalize loss to account for batch accumulation
        #output.loss = output.loss / accum_iter

        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        
        #if ((idx + 1) % accum_iter == 0) or (idx + 1 == len(train_loader)):
        optimizer.step()
        ##### TODO: Apply linear learning rate decay #####
        scheduler.step()
        optimizer.zero_grad()
        
        step += 1
        
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, doc_stride, dev_paragraphs[dev_questions[i]['paragraph_id']],
                    dev_paragraphs_tokenized[dev_questions[i]['paragraph_id']].tokens) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model/macbert6" 
model.save_pretrained(model_save_dir)

total_steps 11424
Start Training ...


Epoch 1 | Step 100 | loss = 2.667, acc = 0.322
Epoch 1 | Step 200 | loss = 0.974, acc = 0.647
Epoch 1 | Step 300 | loss = 0.793, acc = 0.683
Epoch 1 | Step 400 | loss = 0.650, acc = 0.720
Epoch 1 | Step 500 | loss = 0.669, acc = 0.736
Epoch 1 | Step 600 | loss = 0.653, acc = 0.749
Epoch 1 | Step 700 | loss = 0.596, acc = 0.757
Epoch 1 | Step 800 | loss = 0.634, acc = 0.744
Epoch 1 | Step 900 | loss = 0.621, acc = 0.752
Epoch 1 | Step 1000 | loss = 0.519, acc = 0.781
Epoch 1 | Step 1100 | loss = 0.554, acc = 0.770
Epoch 1 | Step 1200 | loss = 0.571, acc = 0.775
Epoch 1 | Step 1300 | loss = 0.549, acc = 0.785
Epoch 1 | Step 1400 | loss = 0.598, acc = 0.755
Epoch 1 | Step 1500 | loss = 0.530, acc = 0.780
Epoch 1 | Step 1600 | loss = 0.607, acc = 0.738
Epoch 1 | Step 1700 | loss = 0.571, acc = 0.772
Epoch 1 | Step 1800 | loss = 0.505, acc = 0.791
Epoch 1 | Step 1900 | loss = 0.563, acc = 0.796
Epoch 1 | Step 2000 | loss = 0.531, acc = 0.809
Epoch 1 | Step 2100 | loss = 0.551, acc = 0.776
E

Epoch 2 | Step 100 | loss = 0.257, acc = 0.873
Epoch 2 | Step 200 | loss = 0.251, acc = 0.890
Epoch 2 | Step 300 | loss = 0.273, acc = 0.861
Epoch 2 | Step 400 | loss = 0.265, acc = 0.866
Epoch 2 | Step 500 | loss = 0.248, acc = 0.879
Epoch 2 | Step 600 | loss = 0.280, acc = 0.873
Epoch 2 | Step 700 | loss = 0.270, acc = 0.873
Epoch 2 | Step 800 | loss = 0.240, acc = 0.874
Epoch 2 | Step 900 | loss = 0.263, acc = 0.873
Epoch 2 | Step 1000 | loss = 0.288, acc = 0.847
Epoch 2 | Step 1100 | loss = 0.241, acc = 0.880
Epoch 2 | Step 1200 | loss = 0.276, acc = 0.877
Epoch 2 | Step 1300 | loss = 0.253, acc = 0.881
Epoch 2 | Step 1400 | loss = 0.265, acc = 0.884
Epoch 2 | Step 1500 | loss = 0.272, acc = 0.857
Epoch 2 | Step 1600 | loss = 0.242, acc = 0.864
Epoch 2 | Step 1700 | loss = 0.221, acc = 0.884
Epoch 2 | Step 1800 | loss = 0.297, acc = 0.864
Epoch 2 | Step 1900 | loss = 0.287, acc = 0.866
Epoch 2 | Step 2000 | loss = 0.267, acc = 0.875
Epoch 2 | Step 2100 | loss = 0.231, acc = 0.879
E

Epoch 3 | Step 100 | loss = 0.135, acc = 0.930
Epoch 3 | Step 200 | loss = 0.133, acc = 0.925
Epoch 3 | Step 300 | loss = 0.118, acc = 0.930
Epoch 3 | Step 400 | loss = 0.130, acc = 0.933
Epoch 3 | Step 500 | loss = 0.125, acc = 0.933
Epoch 3 | Step 600 | loss = 0.151, acc = 0.924
Epoch 3 | Step 700 | loss = 0.092, acc = 0.949
Epoch 3 | Step 800 | loss = 0.119, acc = 0.941
Epoch 3 | Step 900 | loss = 0.130, acc = 0.929
Epoch 3 | Step 1000 | loss = 0.117, acc = 0.935
Epoch 3 | Step 1100 | loss = 0.124, acc = 0.931
Epoch 3 | Step 1200 | loss = 0.135, acc = 0.921
Epoch 3 | Step 1300 | loss = 0.100, acc = 0.941
Epoch 3 | Step 1400 | loss = 0.121, acc = 0.938
Epoch 3 | Step 1500 | loss = 0.102, acc = 0.947
Epoch 3 | Step 1600 | loss = 0.116, acc = 0.933
Epoch 3 | Step 1700 | loss = 0.136, acc = 0.926
Epoch 3 | Step 1800 | loss = 0.128, acc = 0.922
Epoch 3 | Step 1900 | loss = 0.118, acc = 0.934
Epoch 3 | Step 2000 | loss = 0.111, acc = 0.946
Epoch 3 | Step 2100 | loss = 0.120, acc = 0.930
E

## Testing

In [16]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, doc_stride, test_paragraphs[test_questions[i]['paragraph_id']],
                               test_paragraphs_tokenized[test_questions[i]['paragraph_id']].tokens))

result_file = "/content/gdrive/MyDrive/result_macbert6.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


found [UNK] in prediction, using original text
original prediction 溥[UNK]
final prediction 溥儁
found [UNK] in prediction, using original text
original prediction 目前沒有觀察到任何語言純[UNK]以力道來區分不同輔音
final prediction 目前沒有觀察到任何語言純綷以力道來區分不同輔音
found [UNK] in prediction, using original text
original prediction [UNK]人國
final prediction 荇人國
found [UNK] in prediction, using original text
original prediction 馬[UNK]
final prediction 馬馼
found [UNK] in prediction, using original text
original prediction 東晉常[UNK]
final prediction 東晉常璩
found [UNK] in prediction, using original text
original prediction [UNK]稻
final prediction 秈稻
found [UNK] in prediction, using original text
original prediction 白[UNK]紀滅絕事件
final prediction 白堊紀滅絕事件
found [UNK] in prediction, using original text
original prediction 抗佝[UNK]病
final prediction 抗佝僂病
found [UNK] in prediction, using original text
original prediction 杭州[UNK]橋機場
final prediction 杭州筧橋機場
found [UNK] in prediction, using original text
original prediction 蔡[UNK]
final pred

## Kaggle puclic: 0.84610

# Reference
1. I modify the sample code from NTU machine learning course